In [ ]:

TOPIC: Data Warehousing Fundamentals
1. Design a data warehouse schema for a retail company that includes dimension tables for products, customers, and time. Implement the schema using a relational database management system (RDBMS) of your choice.

Fact Table: Sales Fact Table
Dimension Tables: Product Dimension, Customer Dimension, Time Dimension
Schema:

Sales Fact Table:

sale_id: INT (Primary Key)
product_id: INT (Foreign Key to Product Dimension)
customer_id: INT (Foreign Key to Customer Dimension)
date_id: INT (Foreign Key to Time Dimension)
sales_amount: DECIMAL
Product Dimension:

product_id: INT (Primary Key)
product_name: VARCHAR
category: VARCHAR
price: DECIMAL
Customer Dimension:

customer_id: INT (Primary Key)
customer_name: VARCHAR
address: VARCHAR
email: VARCHAR
Time Dimension:

date_id: INT (Primary Key)
date: DATE
day: INT
month: INT
year: INT
quarter: INT

SQL to Create Tables (Using PostgreSQL):

-- Product Dimension Table
CREATE TABLE product_dim (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(100),
    category VARCHAR(50),
    price DECIMAL(10, 2)
);

-- Customer Dimension Table
CREATE TABLE customer_dim (
    customer_id SERIAL PRIMARY KEY,
    customer_name VARCHAR(100),
    address VARCHAR(200),
    email VARCHAR(100)
);

-- Time Dimension Table
CREATE TABLE time_dim (
    date_id SERIAL PRIMARY KEY,
    date DATE,
    day INT,
    month INT,
    year INT,
    quarter INT
);

-- Sales Fact Table
CREATE TABLE sales_fact (
    sale_id SERIAL PRIMARY KEY,
    product_id INT REFERENCES product_dim(product_id),
    customer_id INT REFERENCES customer_dim(customer_id),
    date_id INT REFERENCES time_dim(date_id),
    sales_amount DECIMAL(10, 2)
);

   2. Create a fact table that captures sales data, including product ID, customer ID, date, and sales amount. Populate the fact table with sample data.

   -- Insert data into product_dim
INSERT INTO product_dim (product_name, category, price)
VALUES ('Laptop', 'Electronics', 1000.00),
       ('Shoes', 'Apparel', 50.00);

-- Insert data into customer_dim
INSERT INTO customer_dim (customer_name, address, email)
VALUES ('John Doe', '123 Main St', 'john@example.com'),
       ('Jane Smith', '456 Oak St', 'jane@example.com');

-- Insert data into time_dim
INSERT INTO time_dim (date, day, month, year, quarter)
VALUES ('2023-08-01', 1, 8, 2023, 3),
       ('2023-08-02', 2, 8, 2023, 3);

-- Insert data into sales_fact
INSERT INTO sales_fact (product_id, customer_id, date_id, sales_amount)
VALUES (1, 1, 1, 1000.00),
       (2, 2, 2, 50.00);

   3. Write SQL queries to retrieve sales data from the data warehouse, including aggregations and filtering based on different dimensions.

Total Sales by Product:

SELECT p.product_name, SUM(s.sales_amount) AS total_sales
FROM sales_fact s
JOIN product_dim p ON s.product_id = p.product_id
GROUP BY p.product_name;

Total Sales by Customer:

SELECT c.customer_name, SUM(s.sales_amount) AS total_sales
FROM sales_fact s
JOIN customer_dim c ON s.customer_id = c.customer_id
GROUP BY c.customer_name;

Sales in August 2023:
SELECT t.month, t.year, SUM(s.sales_amount) AS total_sales
FROM sales_fact s
JOIN time_dim t ON s.date_id = t.date_id
WHERE t.month = 8 AND t.year = 2023
GROUP BY t.month, t.year;

TOPIC: ETL and Data Integration
  1. Design an ETL process using a programming language (e.g., Python) to extract data from a source system (e.g., CSV files), transform it by applying certain business rules or calculations, and load it into a data warehouse.
  TOPIC: ETL and Data Integration

  import pandas as pd
import psycopg2

def extract_data(csv_file):
    # Extract data from CSV file
    data = pd.read_csv(csv_file)
    return data

def transform_data(data):
    # Apply transformations like removing duplicates, handling missing values
    data = data.drop_duplicates()
    data = data.dropna()
    return data

def load_data(data, connection):
    cursor = connection.cursor()

    # Insert data into the sales_fact table
    for index, row in data.iterrows():
        cursor.execute("""
            INSERT INTO sales_fact (product_id, customer_id, date_id, sales_amount)
            VALUES (%s, %s, %s, %s)
        """, (row['product_id'], row['customer_id'], row['date_id'], row['sales_amount']))

    connection.commit()

def etl_process(csv_file, connection):
    data = extract_data(csv_file)
    transformed_data = transform_data(data)
    load_data(transformed_data, connection)

# Usage
connection = psycopg2.connect(
    host="localhost",
    database="retail_db",
    user="user",
    password="password"
)

etl_process("sales_data.csv", connection)
connection.close()

1. Design an ETL Process (Python)

Extract: Extract data from a CSV file.
Transform: Apply business rules like converting data formats, filtering invalid data.
Load: Load data into the data warehouse.
ETL Design:

Source: CSV file containing product sales data.
Transformations: Convert data types, remove duplicates, validate fields.
Load: Insert data into the sales_fact table.


  2. Implement the ETL process by writing code that performs the extraction, transformation, and loading steps.
  import pandas as pd
import psycopg2

def extract_data(csv_file):
    # Extract data from CSV file
    data = pd.read_csv(csv_file)
    return data

def transform_data(data):
    # Apply transformations like removing duplicates, handling missing values
    data = data.drop_duplicates()
    data = data.dropna()
    return data

def load_data(data, connection):
    cursor = connection.cursor()

    # Insert data into the sales_fact table
    for index, row in data.iterrows():
        cursor.execute("""
            INSERT INTO sales_fact (product_id, customer_id, date_id, sales_amount)
            VALUES (%s, %s, %s, %s)
        """, (row['product_id'], row['customer_id'], row['date_id'], row['sales_amount']))

    connection.commit()

def etl_process(csv_file, connection):
    data = extract_data(csv_file)
    transformed_data = transform_data(data)
    load_data(transformed_data, connection)

# Usage
connection = psycopg2.connect(
    host="localhost",
    database="retail_db",
    user="user",
    password="password"
)

etl_process("sales_data.csv", connection)
connection.close()

TOPIC: Dimensional Modeling and Schemas

1. Design a Star Schema for University Database

Fact Table: Enrollment Fact

enrollment_id: INT
student_id: INT (FK)
course_id: INT (FK)
date_id: INT (FK)
grade: CHAR(2)
Student Dimension:

student_id: INT (Primary Key)
student_name: VARCHAR
major: VARCHAR
Course Dimension:

course_id: INT (Primary Key)
course_name: VARCHAR
department: VARCHAR
Time Dimension:

date_id: INT (Primary Key)
semester: VARCHAR
year: INT
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   -- Fact Table: Enrollment
CREATE TABLE enrollment_fact (
    enrollment_id SERIAL PRIMARY KEY,
    student_id INT REFERENCES student_dim(student_id),
    course_id INT REFERENCES course_dim(course_id),
    date_id INT REFERENCES time_dim(date_id),
    grade CHAR(2)
);

-- Student Dimension
CREATE TABLE student_dim (
    student_id SERIAL PRIMARY KEY,
    student_name VARCHAR(100),
    major VARCHAR(50)
);

-- Course Dimension
CREATE TABLE course_dim (
    course_id SERIAL PRIMARY KEY,
    course_name VARCHAR(100),
    department VARCHAR(50)
);

-- Time Dimension
CREATE TABLE time_dim (
    date_id SERIAL PRIMARY KEY,
    semester VARCHAR(20),
    year INT
);

   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables.

   -- Fact Table: Enrollment
CREATE TABLE enrollment_fact (
    enrollment_id SERIAL PRIMARY KEY,
    student_id INT REFERENCES student_dim(student_id),
    course_id INT REFERENCES course_dim(course_id),
    date_id INT REFERENCES time_dim(date_id),
    grade CHAR(2)
);

-- Student Dimension
CREATE TABLE student_dim (
    student_id SERIAL PRIMARY KEY,
    student_name VARCHAR(100),
    major VARCHAR(50)
);

-- Course Dimension
CREATE TABLE course_dim (
    course_id SERIAL PRIMARY KEY,
    course_name VARCHAR(100),
    department VARCHAR(50)
);

-- Time Dimension
CREATE TABLE time_dim (
    date_id SERIAL PRIMARY KEY,
    semester VARCHAR(20),
    year INT
);


TOPIC: Performance Optimization and Querying
    1. Scenario: You need to improve the performance of your data loading process in the data warehouse. Write a Python script that implements the following optimizations:
Utilize batch processing techniques to load data in bulk instead of individual row insertion.

. Python Script for Optimized Data Loading

Batch Processing: Load data in bulk instead of row-by-row.
Multi-threading/Multiprocessing: Parallelize the loading process.

import psycopg2
import threading
import pandas as pd
from multiprocessing import Pool

# Batch processing
def load_batch(data_batch, connection):
    cursor = connection.cursor()
    for row in data_batch:
        cursor.execute("""
            INSERT INTO sales_fact (product_id, customer_id, date_id, sales_amount)
            VALUES (%s, %s, %s, %s)
        """, (row['product_id'], row['customer_id'], row['date_id'], row['sales_amount']))
    connection.commit()

# Parallel processing using multiprocessing
def parallel_load(data, connection):
    num_workers = 4
    pool = Pool(num_workers)

    # Split data into chunks for parallel loading
    chunks = [data[i::num_workers] for i in range(num_workers)]
    pool.starmap(load_batch, [(chunk, connection) for chunk in chunks])
    pool.close()
    pool.join()

# Measure performance
def measure_performance(csv_file, connection):
    data = pd.read_csv(csv_file).to_dict(orient='records')

    # Load data without optimization
    start_time = time.time()
    load_batch(data, connection)
    print("Time without optimization:", time.time() - start_time)

    # Load data with parallel processing
    start_time = time.time()
    parallel_load(data, connection)
    print("Time with optimization:", time.time() - start_time)

      b)  Implement multi-threading or multiprocessing to parallelize the data loading process.
      import psycopg2
import threading
import time
import pandas as pd

# Function to load a batch of data
def load_batch(data_batch, connection_params):
    try:
        connection = psycopg2.connect(**connection_params)
        cursor = connection.cursor()
        for row in data_batch:
            cursor.execute("""
                INSERT INTO sales_fact (product_id, customer_id, date_id, sales_amount)
                VALUES (%s, %s, %s, %s)
            """, (row['product_id'], row['customer_id'], row['date_id'], row['sales_amount']))
        connection.commit()
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error while loading data batch: {e}")

# Multi-threaded data loading
def multi_threaded_load(data, connection_params, num_threads=4):
    threads = []
    # Split data into chunks for each thread
    chunk_size = len(data) // num_threads
    for i in range(num_threads):
        start_index = i * chunk_size
        if i == num_threads - 1:
            data_chunk = data[start_index:]  # Last chunk takes the remaining data
        else:
            data_chunk = data[start_index:start_index + chunk_size]

        # Create a thread for each chunk
        thread = threading.Thread(target=load_batch, args=(data_chunk, connection_params))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

# Example usage
if __name__ == "__main__":
    connection_params = {
        'host': 'localhost',
        'database': 'retail_db',
        'user': 'your_user',
        'password': 'your_password'
    }
    data = pd.read_csv('sales_data.csv').to_dict(orient='records')

    # Measure time for multi-threaded loading
    start_time = time.time()
    multi_threaded_load(data, connection_params, num_threads=4)
    print(f"Time taken with multi-threading:

import psycopg2
import multiprocessing as mp
import time
import pandas as pd

# Function to load a batch of data (for multiprocessing)
def load_batch_mp(data_batch, connection_params):
    try:
        connection = psycopg2.connect(**connection_params)
        cursor = connection.cursor()
        for row in data_batch:
            cursor.execute("""
                INSERT INTO sales_fact (product_id, customer_id, date_id, sales_amount)
                VALUES (%s, %s, %s, %s)
            """, (row['product_id'], row['customer_id'], row['date_id'], row['sales_amount']))
        connection.commit()
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error while loading data batch: {e}")

# Multiprocessing data loading
def multiprocessing_load(data, connection_params, num_processes=4):
    pool = mp.Pool(num_processes)
    chunk_size = len(data) // num_processes
    chunks = [data[i * chunk_size: (i + 1) * chunk_size] for i in range(num_processes)]

    # Pass the data chunks and connection params to the multiprocessing pool
    pool.starmap(load_batch_mp, [(chunk, connection_params) for chunk in chunks])
    pool.close()
    pool.join()

# Example usage
if __name__ == "__main__":
    connection_params = {
        'host': 'localhost',
        'database': 'retail_db',
        'user': 'your_user',
        'password': 'your_password'
    }
    data = pd.read_csv('sales_data.csv').to_dict(orient='records')

    # Measure time for multiprocessing loading
    start_time = time.time()
    multiprocessing_load(data, connection_params, num_processes=4)
    print(f"Time taken with multiprocessing: {time.time() - start_time} seconds")

      c)  Measure the time taken to load a specific amount of data before and after implementing these optimizations.

      3. Measure the Time Before and After Optimizations
To compare the time taken with and without optimizations:

Without Optimization (single-threaded approach):
if __name__ == "__main__":
    connection_params = {
        'host': 'localhost',
        'database': 'retail_db',
        'user': 'your_user',
        'password': 'your_password'
    }
    data = pd.read_csv('sales_data.csv').to_dict(orient='records')

    # Single-threaded loading
    start_time = time.time()
    load_batch(data, connection_params)
    print(f"Time taken without optimization: {time.time() - start_time} seconds")

With Multi-threading:

if __name__ == "__main__":
    start_time = time.time()
    multi_threaded_load(data, connection_params, num_threads=4)
    print(f"Time taken with multi-threading: {time.time() - start_time} seconds")

With Multiprocessing:

if __name__ == "__main__":
    start_time = time.time()
    multiprocessing_load(data, connection_params, num_processes=4)
    print(f"Time taken with multiprocessing: {time.time() - start_time} seconds")

Expected Results:
You will observe that the single-threaded approach will take the longest time.
The multi-threaded and multiprocessing approaches will significantly reduce the time taken to load the data, with multiprocessing likely being the fastest if your system has multiple CPU cores.
